Example and Explanation of Using Historia
-----------------------------------------------------------

This notebook will walk through how to use Historia on an example Android application.
The process is roughly:
1. Choose a location and safety property in the application
2. Run Historia with no additional CBCFTL specifications
3. Look at the alarm
4. Add CBCFTL specifications to remove the alarm
5. After adding enough sound CBCFTL specifications, we can prove the example

The example we will be using is the motivating example of our paper.  The full compiled app and source code may be found in the `AntennapodPlayerFragment_fix` directory.  However, feel free to modify this notebook and run it on other open source applications.  The only external input to this notebook that is specific to this example is the APK compiled in debug mode.  The CBCFTL specifications used are written below.

In [2]:
// location of the apk under analysis
val inputApk = "/home/notebooks/AntennapodPlayerFragment_fix/app/build/outputs/apk/debug/app-debug.apk"

// a few dependencies for the notebook:
import $ivy.`com.github.pathikrit::better-files:3.9.1`
import $ivy.`com.lihaoyi:ujson_2.13:1.3.8`
import $ivy.`com.lihaoyi::scalatags:0.12.0`

inputApk: String = "/home/notebooks/AntennapodPlayerFragment_fix/app/build/outputs/apk/debug/app-debug.apk"

Choosing a Location and Safety Property
---------------------------------------

For reference, the code we are analyzing is printed by the cell below.

In [19]:
import better.files._

println(File("/home/notebooks/AntennapodPlayerFragment_fix/app/src/main/java/com/example/row1antennapodrxjava/ui/main/PlayerFragment.java").contentAsString
        .split('\n')
        .zipWithIndex // add line numbers
        .filter{case (line,ind) => !line.startsWith("import") && !line.trim.startsWith("//") && line.trim != ""} // remove some clutter
        .map{case (line,ind) => s"${ind + 1}  $line"}
        .mkString("\n")
       ) 

1  package com.example.row1antennapodrxjava.ui.main;
25  public class PlayerFragment extends Fragment implements Action1<Object> {
27      private Subscription sub;
29      public static PlayerFragment newInstance() {
30          return new PlayerFragment();
31      }
33      @Nullable
34      @Override
35      public View onCreateView(@NonNull LayoutInflater inflater, @Nullable ViewGroup container,
36                               @Nullable Bundle savedInstanceState) {
37          return inflater.inflate(R.layout.main_fragment, container, false);
38      }
40      @Override
41      public void onCreate(@Nullable Bundle savedInstanceState) {
42          super.onCreate(savedInstanceState);
43      }
45      @Override
46      public void onActivityCreated(@Nullable Bundle savedInstanceState) {
47          super.onActivityCreated(savedInstanceState);
48          sub = Single.create(a -> {
55          }).subscribeOn(Schedulers.newThread())
56                  .observeOn(AndroidSchedulers.m

import better.files._



The code above is a more complete version of the code in Figure 2 (a).  We would like to prove that dereferencing `act` on line 27 cannot crash.  The first step is to tell Historia what we would like to prove and where it is.  This is done by creating a `InitialQuery`, Specifically a `ReceiverNonNull`.

In [32]:
import edu.colorado.plv.bounder.symbolicexecutor.state.{InitialQuery,ReceiverNonNull}
import edu.colorado.plv.bounder.lifestate.LifeState

// The method signature unambiguously identifies the method in the application
val methodSignature = LifeState.Signature("com.example.row1antennapodrxjava.ui.main.PlayerFragment",
          "void call(java.lang.Object)")

val initialQuery = ReceiverNonNull(
        methodSignature,
        63, // line number in source code file
        Some(".*toString.*") // regular expression matching receiver (in case multiple dereferences on one line)
    )

import edu.colorado.plv.bounder.symbolicexecutor.state.{InitialQuery,ReceiverNonNull}

import edu.colorado.plv.bounder.lifestate.LifeState

// The method signature unambiguously identifies the method in the application

methodSignature: LifeState.Signature = Signature(
  "com.example.row1antennapodrxjava.ui.main.PlayerFragment",
  "void call(java.lang.Object)"
)
initialQuery: ReceiverNonNull = ReceiverNonNull(
  Signature(
    "com.example.row1antennapodrxjava.ui.main.PlayerFragment",
    "void call(java.lang.Object)"
  ),
  63,
  Some(".*toString.*")
)

Run Historia With No Additional CBCFTL Specifications
-----------------------------------------------------

Next we run historia with no constraints on what the framework may do.  In many cases, an app can be proven safe before adding CBCFTL specifications.  If it cannot, the counter example is useful for writing the CBCFTL.

In [33]:
// load historia code

val jarpath = s"/home/bounder/target/scala-2.13/soot_hopper-assembly-0.1.jar"
interp.load.cp(os.Path(jarpath))

import edu.colorado.plv.bounder.{Driver,RunConfig, BounderUtil} // Historia utilities
import upickle.default.read
import upickle.default.write

// define a function to call the JAR implementation of Historia with a configuration
// If changes are made to Historia, run "sbt compile" in the /home/implementation directory to regenerate the Historia JAR

def runHistoriaWithSpec(configPath:File):Driver.LocResult = {
    val javaMemLimit=8 // Gb Note that this only limits JVM not JNI which can go significantly higher
    val historiaJar = "/home/bounder/target/scala-2.13/soot_hopper-assembly-0.1.jar"
    val apkRootDir = "/home/historia_generalizability"
    val outDir = configPath.parent.toString
    val config = read[RunConfig](configPath.contentAsString)
    val outSubdir = config.outFolder.get.replace("${baseDirOut}",outDir)
    val cmd = s"java -Xmx${javaMemLimit}G -jar ${historiaJar} -m verify -c ${configPath} -b ${apkRootDir} -u ${outDir} -o MEM --debug"
    BounderUtil.runCmdStdout(cmd)

    read[Driver.LocResult]((outSubdir / "result_0.txt").contentAsString)
}



jarpath: String = "/home/bounder/target/scala-2.13/soot_hopper-assembly-0.1.jar"
import edu.colorado.plv.bounder.{Driver,RunConfig, BounderUtil} // Historia utilities

import upickle.default.read

import upickle.default.write

// define a function to call the JAR implementation of Historia with a configuration
// If changes are made to Historia, run "sbt compile" in the /home/implementation directory to regenerate the Historia JAR


defined function runHistoriaWithSpec

Next we create the `RunConfig` that specifies things like the APK, the output folder, the initial query, and the CBCFTL.  This is all written to a `.json` file used later.

In [34]:
val outputDir = File("/home/notebooks/ExampleOut")
val cfg =  RunConfig(apkPath = inputApk.toString, 
          outFolder = Some(outputDir.toString),
          initialQuery = List(initialQuery))

val cfgPath = (outputDir / "cfg.json")
cfgPath.overwrite(write(cfg))  

outputDir: File = /home/notebooks/ExampleOut
cfg: RunConfig = RunConfig(
  "/home/notebooks/AntennapodPlayerFragment_fix/app/build/outputs/apk/debug/app-debug.apk",
  Some("/home/notebooks/ExampleOut"),
  None,
  TopSpecSet,
  List(
    ReceiverNonNull(
      Signature(
        "com.example.row1antennapodrxjava.ui.main.PlayerFragment",
        "void call(java.lang.Object)"
      ),
      63,
      Some(".*toString.*")
    )
  ),
  -1,
  5,
  ExpTag("", "", ""),
  600,
  true,
  None
)
cfgPath: File = /home/notebooks/ExampleOut/cfg.json
res33_3: File = /home/notebooks/ExampleOut/cfg.json

Now we can call Historia.

In [35]:
runHistoriaWithSpec(cfgPath)

res34: Driver.LocResult = LocResult(
  ReceiverNonNull(
    Signature(
      "com.example.row1antennapodrxjava.ui.main.PlayerFragment",
      "void call(java.lang.Object)"
    ),
    63,
    Some(".*toString.*")
  ),
  -1,
  AppLoc(
    SerializedIRMethodLoc(
      "com.example.row1antennapodrxjava.ui.main.PlayerFragment",
      "void call(java.lang.Object)",
      List(
        Some(
          LocalWrapper(
            "@this",
            "com.example.row1antennapodrxjava.ui.main.PlayerFragment"
          )
        ),
        Some(LocalWrapper("@parameter0", "java.lang.Object"))
      )
    ),
    SerializedIRLineLoc(
      63,
      "line: 63 virtualinvoke $r2.<java.lang.Object: java.lang.String toString()>()",
      9848877
    ),
    true
  ),
  Witnessed,
  MultiCallback,
  1L,
  DepthResult(9, 1, 1, Witnessed),
  List(
    List(
      "witnessed",
...

Look at the Alarm
-----------------
Next we look at the alarm that was found.  In this case, it finds the initial state just before the `call` callback.  The witness also shows that `getActivity` was invoked, this is important because it is where a null value may come from.

In [38]:
println(File("/home/notebooks/ExampleOut/wit.witnesses").contentAsString)

witnessed
    WITNESSED: [CB Inv] com.example.row1antennapodrxjava.ui.main.PlayerFragment void call(java.lang.Object)
       state: (    heap:           pure:          types: List()    trace: )
    pre-line: 62 $r2 = virtualinvoke r0.<com.example.row1antennapodrxjava.ui.main.PlayerFragment: androidx.fragment.app.FragmentActivity getActivity()>()
       state: ( locals:          heap:           pure:          types: List()    trace: )
    [CI Inv merge] androidx.fragment.app.FragmentActivity getActivity()
       state: ([CI Ret] androidx.fragment.app.Fragment androidx.fragment.app.FragmentActivity getActivity() locals:       locals:          heap:           pure:          types: List()    trace: )
    [CI Ret merge] androidx.fragment.app.FragmentActivity getActivity()
       state: ([CI Ret] androidx.fragment.app.Fragment androidx.fragment.app.FragmentActivity getActivity() locals:       locals:          heap:           pure:          types: List()    trace: )



Add CBCFTL Specifications to Remove the Alarm
--------------------------

Writing a CBCFTL specification consists of looking at counter examples like the one above and explaining when the framework can *NOT* do something like return a `null` value.  An easy, yet unsound, CBCFTL specification is to say `getActivity` cannot return null ever.  However, writing this specification is a useful step to show how CBCFTL specifications are constructed.  We will write a sound specification later.

We write that as `null = cb a.getActivity() -[]-> false` in the paper.  Below we show how to write this same thing for the implementation.

In [39]:

import edu.colorado.plv.bounder.lifestate.LifeState.{LSSpec, LSFalse,LSConstraint}
import edu.colorado.plv.bounder.symbolicexecutor.state.{NamedPureVar,NullVal,Equals}
import edu.colorado.plv.bounder.lifestate.SpecSignatures

val a = NamedPureVar("a")  // variables used in spec
val f = NamedPureVar("f")

val getActivityNullUnsound = LSSpec(a::f::Nil, Nil,
    LSFalse, 
    SpecSignatures.Fragment_get_activity_exit, // abstract message f = cb a.getActivity()  (defined in Specifications.scala)
    Set(LSConstraint(a, Equals, NullVal)))

import edu.colorado.plv.bounder.lifestate.LifeState.{LSSpec, LSFalse,LSConstraint}

import edu.colorado.plv.bounder.symbolicexecutor.state.{NamedPureVar,NullVal,Equals}

import edu.colorado.plv.bounder.lifestate.SpecSignatures


a: NamedPureVar = NamedPureVar("a")
f: NamedPureVar = NamedPureVar("f")
getActivityNullUnsound: LSSpec = LSSpec(
  List(NamedPureVar("a"), NamedPureVar("f")),
  List(),
  False,
  OAbsMsg(
    CIExit,
    SubClassMatcher(
      Set(
        "android.app.Fragment",
        "androidx.fragment.app.Fragment",
        "android.support.v4.app.Fragment"
      ),
      ".*Activity getActivity\\(\\)",
      "Fragment_getActivity"
    ),
    List(NamedPureVar("a"), NamedPureVar("f"))
  ),
  Set(LSConstraint(NamedPureVar("a"),  == , NULL))
)

In [1]:
val userhome = System.getProperty("user.home")
val jniPath = s"${userhome}/software/z3/build"
val newPath = Array(jniPath) ++  System.getProperty("java.library.path").split(":")
System.setProperty("java.library.path",newPath.distinct.mkString(":"))
val sysPathsField = classOf[ClassLoader].getDeclaredField("sys_paths");
sysPathsField.setAccessible(true);
sysPathsField.set(null, null);

import $ivy.`org.plotly-scala:plotly-almond_2.13:0.8.2`
import $ivy.`com.github.pathikrit::better-files:3.9.1`
import $ivy.`com.github.nscala-time::nscala-time:2.32.0`

import $ivy.`com.lihaoyi:ujson_2.13:1.3.8`
// import $ivy.`edu.colorado.plv.bounder:soot_hopper_2.13:0.1`

import plotly._, element._, layout._, Plotly._
import ujson.Value
import sys.process._



import jupyter.Displayer, jupyter.Displayers
import scala.collection.JavaConverters._
import scala.collection.mutable
import com.github.nscala_time.time.Imports._
import org.joda.time.Period

userhome: String = "/root"
jniPath: String = "/root/software/z3/build"
newPath: Array[String] = Array(
  "/root/software/z3/build",
  "/usr/lib/",
  "/usr/java/packages/lib",
  "/usr/lib64",
  "/lib64",
  "/lib",
  "/usr/lib"
)
res0_3: String = "/usr/lib/:/usr/java/packages/lib:/usr/lib64:/lib64:/lib:/usr/lib"
sysPathsField: java.lang.reflect.Field = private static java.lang.String[] java.lang.ClassLoader.sys_paths
jarpath: String = "/home/bounder/target/scala-2.13/soot_hopper-assembly-0.1.jar"
import $ivy.$                                          

import $ivy.$                                         

import $ivy.$                                           


import $ivy.$                             
// import $ivy.`edu.colorado.plv.bounder:soot_hopper_2.13:0.1`


import plotly._, element._, layout._, Plotly._

import ujson.Value

import sys.process._


import $ivy.$                              


import jupyter.Displayer, jupyter.Displayers

import scala.collection.JavaConverter

In [2]:
import better.files._

import scala.util.Random
import edu.colorado.plv.bounder.{Driver,ExpTag,Action,RunConfig,PickleSpec}
import edu.colorado.plv.bounder.lifestate.{SpecSpace,ViewSpec,SpecSignatures}
import edu.colorado.plv.bounder.symbolicexecutor.state.{InitialQuery,Reachable,ReceiverNonNull, DisallowedCallin}

import scala.collection.parallel.CollectionConverters.{ImmutableSetIsParallelizable, IterableIsParallelizable}
import edu.colorado.plv.bounder.ExperimentsDb
import edu.colorado.plv.bounder.BounderUtil
//import scala.concurrent.duration._
import scala.language.postfixOps
import slick.driver.H2Driver.api._
import slick.jdbc.GetResult
import slick.jdbc.SQLActionBuilder
import scala.concurrent.Await
import almond.interpreter.api.DisplayData
import edu.colorado.plv.bounder.ir.Messages


Driver.setZ3Path(s"${userhome}/software/z3/build")

// var android_home_possible = List(s"${userhome}/Library/Android/sdk", s"${userhome}/Android/Sdk")
// var android_home = android_home_possible.find(p => File(p).exists()).get             
// BounderUtil.setEnv(Map("DYLD_LIBRARY_PATH" -> s"${userhome}/software/z3/build","ANDROID_HOME" -> android_home,"HOME" -> userhome))

System.setProperty("user.dir", s"${System.getProperty("user.home")}/Documents/source/bounder/notebooks/ossExp/SpecGen");
val expDir = File("/home/notebooks")

java.library.path set to: /root/software/z3/build:/usr/lib/:/usr/java/packages/lib:/usr/lib64:/lib64:/lib:/usr/lib


import better.files._


import scala.util.Random

import edu.colorado.plv.bounder.{Driver,ExpTag,Action,RunConfig,PickleSpec}

import edu.colorado.plv.bounder.lifestate.{SpecSpace,ViewSpec,SpecSignatures}

import edu.colorado.plv.bounder.symbolicexecutor.state.{InitialQuery,Reachable,ReceiverNonNull, DisallowedCallin}

import upickle.default.read

import upickle.default.write

import scala.collection.parallel.CollectionConverters.{ImmutableSetIsParallelizable, IterableIsParallelizable}

import edu.colorado.plv.bounder.ExperimentsDb

import edu.colorado.plv.bounder.BounderUtil
//import scala.concurrent.duration._

import scala.language.postfixOps

import slick.driver.H2Driver.api._

import slick.jdbc.GetResult

import slick.jdbc.SQLActionBuilder

import scala.concurrent.Await

import almond.interpreter.api.DisplayData

import edu.colorado.plv.bounder.ir.Messages



res1_18: String = "/home/notebooks"
expDir: File = /home/notebooks